In [166]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import re

In [223]:
df = pd.read_csv('./CSV/HISTORICO_PRODUCTOS.csv')
df_org = df.copy(True)
df

,CG_PROD,DES_PROD,CANT,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:43.360,NaN,60603,NaN,NaN,1756.25
1,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:43.987,NaN,60604,NaN,NaN,1756.25
2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:28:07.730,NaN,60643,NaN,NaN,2262.56
3,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:28:08.310,NaN,60644,NaN,NaN,2262.56
4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:59.260,NaN,60629,NaN,NaN,2005.02
...,...,...,...,...,...,...,...,...,...,...
3505,2142H2R1RAM1013,"2100 2""H3"" 300(RF)x150(RF)",1.0,2019-10-17,2020-01-10 11:04:06.967,2020-01-24 00:00:00.000,55960,NaN,NaN,1503.36
3506,2142J1R1RAM1012,"2100 2""J3"" 150(RF)x150(RF)",1.0,2019-10-17,2020-01-13 08:15:30.187,2020-01-24 00:00:00.000,55955,NaN,NaN,1384.32
3507,2142J2R1RAM1012,"2100 2""J3"" 300(RF)x150(RF)",1.0,2019-10-17,2020-01-10 11:04:12.380,2020-01-28 13:54:49.903,55957,NaN,NaN,1398.72
3508,2842J1R1RFM1010,"2800 2""J3"" 150(RF)x150(RF) Con Palanca",1.0,2019-03-06,2020-01-28 09:26:21.813,NaN,56407,NaN,NaN,0.00


In [224]:
df['FE_EMIT'] = df['FE_EMIT'].str[:11]

In [225]:
def date_to_num(date:str):
    year = int(date[:4])
    month = int(date[5:7])
    day = int(date[8:])
    return year*365+month*30+day

In [226]:
df['FE_ENTREGA'] = df['FE_ENTREGA'].apply(lambda x: date_to_num(x))
df['FE_EMIT'] = df['FE_EMIT'].apply(lambda x: date_to_num(x))

### LÓGICA

Si FE_EMIT1<FE_ENTREGA2<FE_ENTREGA1 el producto 2 existió en simúltaneo con el producto 1  

In [227]:
def simultaneo(fe_emit1:int, fe_entrega1:int, fe_emit2:int, fe_entrega2:int):
    return (fe_emit1<=fe_emit2 and fe_emit2<fe_entrega1) or (fe_emit1<fe_entrega2 and fe_entrega2<fe_entrega1) or (fe_emit1>=fe_emit2 and fe_entrega1<=fe_entrega2)

In [228]:
empty = np.full(len(df), np.nan)
df.insert(3, 'SIMULTANEO', empty)

In [229]:
df = df.drop_duplicates('DES_PROD')
df.loc[:, 'SIMULTANEO'] = df.loc[:, 'SIMULTANEO'].apply(lambda x: np.full(len(df), np.nan))
df

C:\Users\mrulli\AppData\Local\Temp\ipykernel_5644\1408353255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'SIMULTANEO'] = df.loc[:, 'SIMULTANEO'].apply(lambda x: np.full(len(df), np.nan))


,CG_PROD,DES_PROD,CANT,SIMULTANEO,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",738759,738513,NaN,60603,NaN,NaN,1756.25
2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",738759,738513,NaN,60643,NaN,NaN,2262.56
4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",738759,738513,NaN,60629,NaN,NaN,2005.02
10,2838H1R1RSM1014,"2800 1.1/2""H3"" 150(RF)x150(RF) sellada",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",738742,738597,NaN,61272,NaN,NaN,1480.80
14,2651K1R1RBM1016,"2600 3""K4"" 150(RF)x150(RF)",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",738718,738534,NaN,60765,NaN,NaN,3355.80
...,...,...,...,...,...,...,...,...,...,...,...
3467,152507M7HSM2519,"1500 1""x1"" BSP(M)xBSP(H) sellada",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",737358,737344,NaN,56366,NaN,NaN,348.00
3473,2142H2R1RAM1013,"2100 2""H3"" 300(RF)x150(RF)",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",737353,737353,2020-01-24 00:00:00.000,55959,NaN,NaN,1503.36
3475,2142J2R1RAM1012,"2100 2""J3"" 300(RF)x150(RF)",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",737353,737353,2020-01-24 00:00:00.000,55958,NaN,NaN,1398.72
3477,2128E3R1RAM1013,"2100 1""E2"" 600(RF)x150(RF)",1.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",737352,737352,2020-01-24 00:00:00.000,55962,NaN,NaN,1158.72


In [230]:
for i in range(len(df)):
    row1 = df.iloc[i]
    for k in range(len(df)):
        row2 = df.iloc[k]
        df.iloc[i][3][k] = simultaneo(row1[5], row1[4], row2[5], row2[4])

In [231]:
np.sum([i for i in df.iloc[0][3]])

164.0

In [ ]:
simultaneo(df.iloc[0][5], df.iloc[0][4], df.iloc[1][5], df.iloc[1][4])

True

In [220]:
simultaneo(df.iloc[0][5], df.iloc[0][4], df.iloc[3][5], df.iloc[3][4])

True

In [232]:
df[np.asarray([i for i in df.iloc[0][3]], dtype=bool)]

,CG_PROD,DES_PROD,CANT,SIMULTANEO,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60603,NaN,NaN,1756.25
2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60643,NaN,NaN,2262.56
4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60629,NaN,NaN,2005.02
10,2838H1R1RSM1014,"2800 1.1/2""H3"" 150(RF)x150(RF) sellada",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738742,738597,NaN,61272,NaN,NaN,1480.80
14,2651K1R1RBM1016,"2600 3""K4"" 150(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738718,738534,NaN,60765,NaN,NaN,3355.80
...,...,...,...,...,...,...,...,...,...,...,...
607,2838F4R2RAM1017,"2800 1.1/2""F3"" 900(RF)x300(RF)",1.0,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",738519,738461,2023-04-11 13:52:21.773,60278,2023-04-11 07:10:47.777,2023-04-11 13:52:01.487,3503.00
608,1617A1R1RSM1010,"1600 1/2"" x 1"" 150(RF) x 150(RF) Tob 1/2""Std. ...",1.0,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",738518,738477,NaN,60369,NaN,NaN,895.40
609,1621B1R1RSM1010,"1600 3/4"" x 1"" 150(RF) x 150(RF)",1.0,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",738518,738477,NaN,60368,NaN,NaN,966.90
610,2842J1R1RSM1010,"2800 2""J3"" 150(RF)x150(RF) sellada",1.0,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",738518,738477,NaN,60370,NaN,NaN,1996.50


In [246]:
df.drop(index=(df[df['FE_EMIT']==df['FE_ENTREGA']].index), axis=1)

,CG_PROD,DES_PROD,CANT,SIMULTANEO,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60603,NaN,NaN,1756.25
2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60643,NaN,NaN,2262.56
4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60629,NaN,NaN,2005.02
10,2838H1R1RSM1014,"2800 1.1/2""H3"" 150(RF)x150(RF) sellada",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738742,738597,NaN,61272,NaN,NaN,1480.80
14,2651K1R1RBM1016,"2600 3""K4"" 150(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738718,738534,NaN,60765,NaN,NaN,3355.80
...,...,...,...,...,...,...,...,...,...,...,...
3422,1827X8M7HAT1310,"1800 1""x1.1/2"" BSPT(M)xBSP(H) asiento metal/te...",1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",737392,737344,2020-03-03 00:00:00.000,56382,NaN,NaN,448.80
3424,1721E9M9HSB1226,"1700 3/4""x1"" NPT(M)xNPT(H) Tob 1"" Std. sello B...",1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",737388,737360,2020-03-16 00:00:00.000,56387,NaN,NaN,720.00
3427,1727D9M9HAM1325,"1700 1""x1.1/2"" NPT(M)xNPT(H) Tob1"" Std.",1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",737388,737336,2020-03-16 00:00:00.000,56284,NaN,NaN,593.00
3428,2136E5J2RAM1013,"2100 1.1/2""E2"" 1500(RJ)x300(RF)",1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",737388,737345,2020-04-30 16:33:58.727,56285,2023-07-11 14:00:00.000,2023-07-11 14:00:00.000,2464.00


In [243]:
len(df)

501

In [82]:
df.to_csv('HISTORICO_SIMUL.csv')

In [100]:
df_org.drop_duplicates('PEDIDO')

,CG_PROD,DES_PROD,CANT,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:43.360,NaN,60603,NaN,NaN,1756.25
1,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:43.987,NaN,60604,NaN,NaN,1756.25
2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:28:07.730,NaN,60643,NaN,NaN,2262.56
3,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:28:08.310,NaN,60644,NaN,NaN,2262.56
4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:59.260,NaN,60629,NaN,NaN,2005.02
...,...,...,...,...,...,...,...,...,...,...
3505,2142H2R1RAM1013,"2100 2""H3"" 300(RF)x150(RF)",1.0,2019-10-17,2020-01-10 11:04:06.967,2020-01-24 00:00:00.000,55960,NaN,NaN,1503.36
3506,2142J1R1RAM1012,"2100 2""J3"" 150(RF)x150(RF)",1.0,2019-10-17,2020-01-13 08:15:30.187,2020-01-24 00:00:00.000,55955,NaN,NaN,1384.32
3507,2142J2R1RAM1012,"2100 2""J3"" 300(RF)x150(RF)",1.0,2019-10-17,2020-01-10 11:04:12.380,2020-01-28 13:54:49.903,55957,NaN,NaN,1398.72
3508,2842J1R1RFM1010,"2800 2""J3"" 150(RF)x150(RF) Con Palanca",1.0,2019-03-06,2020-01-28 09:26:21.813,NaN,56407,NaN,NaN,0.00


In [116]:
df_org.iloc[0], df_org.iloc[500]

(CG_PROD                                         2636F2R1RAM1017
 DES_PROD                         2600 1.1/2"F2" 300(RF)x150(RF)
 CANT                                                        1.0
 FE_ENTREGA                                           2023-12-04
 FE_EMIT                                 2023-03-28 11:27:43.360
 FE_CIERRE                                                   NaN
 PEDIDO                                                    60603
 FECHA_PREVISTA_FABRICACION                                  NaN
 FECHA_INICIO_REAL_FABRICACION                               NaN
 TOT_DOL                                                 1756.25
 Name: 0, dtype: object,
 CG_PROD                                     2666Q1R1RBB3117
 DES_PROD                         2600 6"Q8" 150(RF)x150(RF)
 CANT                                                    1.0
 FE_ENTREGA                                       2023-05-28
 FE_EMIT                             2023-02-28 07:17:56.483
 FE_CIERRE          

In [126]:
df_org['DES_PROD'].str[:4].unique(), df_org['DES_PROD'].str[:7].unique()

(array(['2600', '2800', '1800', 'AC-3', '1700', '2100', '2400', 'AC/R',
        '1600', 'AR-3', 'AR-2', '9300', '1100', '3000', '2805', 'Junt',
        '2405', '1804', '1900', '1500', '1400', 'VA-1', '9200', '9100',
        '1200', 'AR-5', '1805', 'AC-5', 'AR-1', 'VA-3', 'AC-9', '6800',
        'CALE', 'AR-7', 'AR-4', 'PILO', 'ACR-', 'Filt', 'AC-2', 'AR-9'],
       dtype=object),
 array(['2600 1.', '2800 1.', '2600 3"', '1800 1"', '2800 6"', 'AC-350 ',
        '1800 2"', '2800 3"', '1700 3/', '1800 1/', '1800 3/', '2100 1"',
        '2100 1.', '1800 1.', '2800 1"', '1700 1/', '2400 3"', '2800 2"',
        'AC/R-50', '2600 1"', '1700 1"', '1800 AP', '2800 4"', '1600 3/',
        'AR-30 1', 'AR-30 2', 'AR-20 1', '9300 8"', '1100 3/', '9300 6"',
        '3000 VA', '2805 2"', '2400 1.', 'Junta c', '1800 2.', '2405 1.',
        '1600 1"', '2100 4"', '2600 4"', '1804 1"', '1804 1/', '1900 3/',
        '2600 2"', '1500 1/', '1400 1/', 'AR-20 2', 'VA-1000', '9200 3"',
        '9300 4"', '2600 

In [127]:
df.to_csv('HISTORICO_SIMUL_1.csv')

In [162]:
df = pd.read_csv('HISTORICO_SIMUL.csv')
df

,Unnamed: 0,CG_PROD,DES_PROD,CANT,SIMULTANEO,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60603,NaN,NaN,1756.25
1,2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60643,NaN,NaN,2262.56
2,4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60629,NaN,NaN,2005.02
3,10,2838H1R1RSM1014,"2800 1.1/2""H3"" 150(RF)x150(RF) sellada",1.0,[0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738742,738597,NaN,61272,NaN,NaN,1480.80
4,14,2651K1R1RBM1016,"2600 3""K4"" 150(RF)x150(RF)",1.0,[0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738718,738534,NaN,60765,NaN,NaN,3355.80
...,...,...,...,...,...,...,...,...,...,...,...,...
724,3477,2128E3R1RAM1013,"2100 1""E2"" 600(RF)x150(RF)",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737352,737352,2020-01-24 00:00:00.000,55962,NaN,NaN,1158.72
725,3482,1821X9M9HKM1310,"1800 3/4""x1"" NPT(M)xNPT(H) Palanca Sellada",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737345,737345,2020-01-28 00:00:00.000,56269,NaN,NaN,440.00
726,3499,2836F3R1RAM3113,"2800 1.1/2""F2"" 600(RF)x150(RF)",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737318,737318,2020-03-04 00:00:00.000,56238,NaN,NaN,2390.00
727,3500,3515C3Z1,"AC-500 1/2"" BSP H-H Punta eje libre g.indist.",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737318,737318,2020-01-20 00:00:00.000,56244,NaN,NaN,683.00


In [139]:
del df['Unnamed: 0']

In [163]:
df

,Unnamed: 0,CG_PROD,DES_PROD,CANT,SIMULTANEO,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60603,NaN,NaN,1756.25
1,2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60643,NaN,NaN,2262.56
2,4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60629,NaN,NaN,2005.02
3,10,2838H1R1RSM1014,"2800 1.1/2""H3"" 150(RF)x150(RF) sellada",1.0,[0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738742,738597,NaN,61272,NaN,NaN,1480.80
4,14,2651K1R1RBM1016,"2600 3""K4"" 150(RF)x150(RF)",1.0,[0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738718,738534,NaN,60765,NaN,NaN,3355.80
...,...,...,...,...,...,...,...,...,...,...,...,...
724,3477,2128E3R1RAM1013,"2100 1""E2"" 600(RF)x150(RF)",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737352,737352,2020-01-24 00:00:00.000,55962,NaN,NaN,1158.72
725,3482,1821X9M9HKM1310,"1800 3/4""x1"" NPT(M)xNPT(H) Palanca Sellada",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737345,737345,2020-01-28 00:00:00.000,56269,NaN,NaN,440.00
726,3499,2836F3R1RAM3113,"2800 1.1/2""F2"" 600(RF)x150(RF)",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737318,737318,2020-03-04 00:00:00.000,56238,NaN,NaN,2390.00
727,3500,3515C3Z1,"AC-500 1/2"" BSP H-H Punta eje libre g.indist.",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737318,737318,2020-01-20 00:00:00.000,56244,NaN,NaN,683.00
